<a href="https://colab.research.google.com/github/Elmir22/Data-science-ML-models-/blob/main/neural_network_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.linear_model import Perceptron
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import datetime


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()


11490434/11490434 [==============================] - 0s 0us/step


In [3]:
mnist.load_data

<function keras.src.datasets.mnist.load_data(path='mnist.npz')>

In [4]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42)

In [6]:
model = tf.keras.Sequential([
    Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(1024, activation = "relu"),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(256,activation = "relu"),
    tf.keras.layers.Dense(128,activation = "relu"),
    tf.keras.layers.Dense(64,activation = "relu"),
    tf.keras.layers.Dense(10,activation = "softmax")
])
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,metrics=['accuracy'])
mnist_data = model.fit(X_train, y_train, epochs=20,
                    validation_data = (X_val, y_val))

Epoch 1/20
1407/1407 [==============================] - 36s 24ms/step - loss: 2.1564 - accuracy: 0.3260 - val_loss: 1.9100 - val_accuracy: 0.6310
Epoch 2/20
1407/1407 [==============================] - 24s 17ms/step - loss: 1.3939 - accuracy: 0.7347 - val_loss: 0.9007 - val_accuracy: 0.7987
Epoch 3/20
1407/1407 [==============================] - 24s 17ms/step - loss: 0.6867 - accuracy: 0.8345 - val_loss: 0.5530 - val_accuracy: 0.8521
Epoch 4/20
1407/1407 [==============================] - 25s 18ms/step - loss: 0.4845 - accuracy: 0.8726 - val_loss: 0.4349 - val_accuracy: 0.8829
Epoch 5/20
1407/1407 [==============================] - 25s 18ms/step - loss: 0.3986 - accuracy: 0.8918 - val_loss: 0.3753 - val_accuracy: 0.8964
Epoch 6/20
1407/1407 [==============================] - 24s 17ms/step - loss: 0.3501 - accuracy: 0.9038 - val_loss: 0.3385 - val_accuracy: 0.9051
Epoch 7/20
1407/1407 [==============================] - 25s 17ms/step - loss: 0.3182 - accuracy: 0.9096 - val_loss: 0.3104 -

In [7]:
def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(units=hp.Int('units', min_value=128, max_value=1024, step=64), activation='relu'))
    model.add(Dense(10, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [8]:
!pip install -q -U keras-tuner
from keras_tuner import RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='keras_tuner_logs',
    project_name='mnist_hyperparameter_tuning'
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [9]:
tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_val, y_val),
             callbacks=[EarlyStopping('val_accuracy', patience=3),
                        TensorBoard(log_dir='logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))])

Trial 10 Complete [00h 02m 34s]
val_accuracy: 0.9610666632652283

Best val_accuracy So Far: 0.9811333417892456
Total elapsed time: 00h 27m 09s
